In [1]:
import numpy as np
from scene import Scene
import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

scene = Scene(using_cnn=True, init_randomly=True)

pygame 2.1.3 (SDL 2.0.22, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
model = keras.Sequential([
    Conv2D(16, (3, 3), activation='elu', padding="same", kernel_initializer='he_normal', input_shape=(scene.height, scene.width, scene.elements_count)),
    MaxPooling2D(2),
    Conv2D(32, (3, 3), activation='elu', padding="same", kernel_initializer='he_normal'),
    MaxPooling2D(2),
    Flatten(),    
    Dense(16, activation='elu', kernel_initializer='he_normal'),
    Dense(4, activation='linear')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

target = keras.models.clone_model(model)
target.set_weights(model.get_weights())

In [3]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(4)
    else:
        Q_values = model.predict(tf.one_hot(state[np.newaxis], scene.elements_count), verbose=False)
        return np.argmax(Q_values[0])

In [4]:
from collections import deque

# (state, action, reward, next_state, done)
replay_memory = deque(maxlen=2000)

def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_memory), size=batch_size)
    batch = [replay_memory[index] for index in indices]
    states, actions, rewards, next_states, dones = [np.array([experience[field_index]
                                                            for experience in batch])
                                                            for field_index in range(5)]
    return states, actions, rewards, next_states, dones

In [5]:
def play_one_step(scene, state, epsilon):
    action = epsilon_greedy_policy(state, epsilon)
    scene.snake.change_direction(action)
    next_state, reward, done = scene.move()
    replay_memory.append((state, action, reward, next_state, done))
    return next_state, reward, done

In [6]:
batch_size = 32
discount_rate = 0.95
optimizer = keras.optimizers.legacy.Adam(learning_rate=1e-3)
loss_fn = keras.losses.mean_squared_error

def training_step(batch_size):
    states, actions, rewards, next_states, dones = sample_experiences(batch_size)
    next_Q_values = model.predict(tf.one_hot(next_states, scene.elements_count), verbose=False)
    best_next_actions = np.argmax(next_Q_values, axis=1)
    next_mask = tf.one_hot(best_next_actions, 4).numpy()
    next_best_Q_values = (target.predict(tf.one_hot(next_states, scene.elements_count), verbose=False) * next_mask).sum(axis=1)
    target_Q_values = (rewards + (1 - dones) * discount_rate * next_best_Q_values).reshape(-1, 1)
    mask = tf.one_hot(actions, 4)
    with tf.GradientTape() as tape:
        all_Q_values = model(tf.one_hot(states, scene.elements_count))
        Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = tf.reduce_mean(loss_fn(target_Q_values, Q_values))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [7]:
episodes_count = 8000
reward_per_batch = 0

for episode in range(1, episodes_count + 1):
    state = scene.scene_as_matrix()
    epsilon = (1 / (np.linspace(1, 8, episodes_count)**(1/3)))[episode - 1]
    done = False
    
    steps = 0
    while not done:
        steps += 1
        state, reward, done = play_one_step(scene, state, epsilon)
        reward_per_batch += reward
        
    if episode > 50:
        training_step(batch_size)
        if episode % 50 == 0:
            print("Episode number: ", episode)
            target.set_weights(model.get_weights())
            print("Average reward: ", reward_per_batch / 50)
            reward_per_batch = 0
            
    if episode % 1000 == 0:
        model.save(f"models/CNN_model{episode}.h5")

episode:  1
episode:  2
episode:  3
episode:  4
episode:  5
episode:  6
episode:  7
episode:  8
episode:  9
episode:  10
episode:  11
episode:  12
episode:  13
episode:  14
episode:  15
episode:  16
episode:  17
episode:  18
episode:  19
episode:  20
episode:  21
episode:  22
episode:  23
episode:  24
episode:  25
episode:  26
episode:  27
episode:  28
episode:  29
episode:  30
episode:  31
episode:  32
episode:  33
episode:  34
episode:  35
episode:  36
episode:  37
episode:  38
episode:  39
episode:  40
episode:  41
episode:  42
episode:  43
episode:  44
episode:  45
episode:  46
episode:  47
episode:  48
episode:  49
episode:  50
episode:  51
episode:  52
episode:  53
episode:  54
episode:  55
episode:  56
episode:  57
episode:  58
episode:  59
episode:  60
episode:  61
episode:  62
episode:  63
episode:  64
episode:  65
episode:  66
episode:  67
episode:  68
episode:  69
episode:  70
episode:  71
episode:  72
episode:  73
episode:  74
episode:  75
episode:  76
episode:  77
episode:

/Users/juanpablo/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


episode:  202
episode:  203
episode:  204
episode:  205
episode:  206
episode:  207
episode:  208
episode:  209
episode:  210
episode:  211
episode:  212
episode:  213
episode:  214
episode:  215
episode:  216
episode:  217
episode:  218
episode:  219
episode:  220
episode:  221
episode:  222
episode:  223
episode:  224
episode:  225
episode:  226
episode:  227
episode:  228
episode:  229
episode:  230
episode:  231
episode:  232
episode:  233
episode:  234
episode:  235
episode:  236
episode:  237
episode:  238
episode:  239
episode:  240
episode:  241
episode:  242
episode:  243
episode:  244
episode:  245
episode:  246
episode:  247
episode:  248
episode:  249
episode:  250
Episode number:  250
Average reward:  1.02
episode:  251
episode:  252
episode:  253
episode:  254
episode:  255
episode:  256
episode:  257
episode:  258
episode:  259
episode:  260
episode:  261
episode:  262
episode:  263
episode:  264
episode:  265
episode:  266
episode:  267
episode:  268
episode:  269
episo

KeyboardInterrupt: 